In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.datasets.samples_generator import make_blobs

In [30]:
class cluster_node:
    def __init__(self, vec, id, left=None, right=None, distance=0.0, node_vector = None):
        self.leftnode = left
        self.rightnode = right
        self.vec = vec
        self.id = id
        self.distance = distance
        if node_vector is None:
            self.node_vector = [self.id]
        else:
            self.node_vector = node_vector[:]

In [31]:
def euclidean_distance(v1,v2):
    return np.sqrt(sum((v1-v2)**2))

def min_distance(clust1, clust2, distances):
    d = 12123123123123
    for i in clust1.node_vector:
        for j in clust2.node_vector:
            try:
                distance = distances[(i,j)]
            except:
                try:
                    distance = distances[(j,i)]
                except:
                    distance = euclidean_distance(clust1.vec, clust2.vec)
            if distance < d:
                d = distance
    return d


In [32]:
def agglomerative_clustering(data, distance):
    # cluster the rows of the "data" matrix
    distances = {}
    currentclustid = -1

    # clusters are initially just the individual rows
    clust = [cluster_node(np.array(data[i]), id=i) for i in range(len(data))]

    while len(clust) > 1:
        lowestpair = (0,1)
        closest = euclidean_distance(clust[0].vec,clust[1].vec)
    
        # loop through every pair looking for the smallest distance
        for i in range(len(clust)):
            for j in range(i+1,len(clust)):
                # distances is the cache of distance calculations
                if (clust[i].id,clust[j].id) not in distances:
                    if distance == "min":
                        distances[(clust[i].id,clust[j].id)] = min_distance(clust[i], clust[j], distances)
                    else:
                        distances[(clust[i].id,clust[j].id)] = euclidean_distance(clust[i].vec,clust[j].vec)
        
                d = distances[(clust[i].id,clust[j].id)]
        
                if d < closest:
                    closest = d
                    lowestpair = (i,j)
        
        # calculate the average of the two clusters
        len0 = len(clust[lowestpair[0]].node_vector)
        len1 = len(clust[lowestpair[1]].node_vector)
        mean_vector = [(len0*clust[lowestpair[0]].vec[i] + len1*clust[lowestpair[1]].vec[i])/(len0 + len1) \
                        for i in range(len(clust[0].vec))]
        
        # create the new cluster
        newcluster = cluster_node(np.array(mean_vector), currentclustid, left = clust[lowestpair[0]], right = clust[lowestpair[1]], \
            distance = closest, node_vector = clust[lowestpair[0]].node_vector + clust[lowestpair[1]].node_vector)
        
        # cluster ids that weren't in the original set are negative
        currentclustid -= 1
        del clust[lowestpair[1]]
        del clust[lowestpair[0]]
        clust.append(newcluster)

    return clust[0]

In [34]:
#### Now Generate Data
centers = [[1, 1], [-1, -1], [1, -1]]
X, _ = make_blobs(n_samples = 90, centers = centers, cluster_std = 0.1)
df = pd.DataFrame(X)

plt.scatter(df[0],df[1])
plt.show()

In [36]:
data = np.array(df)

### Taking Average
cluster = agglomerative_clustering(data, "avg")
plt.scatter(cluster.leftnode.vec[0], cluster.leftnode.vec[1], color = 'yellow')
plt.scatter(cluster.rightnode.leftnode.vec[0], cluster.rightnode.leftnode.vec[1], color = 'red')
plt.scatter(cluster.rightnode.rightnode.vec[0], cluster.rightnode.rightnode.vec[1], color = 'green')
plt.scatter(df[0],df[1])
plt.show()


## Taking Minimum Distance
# cluster = agglomerative_clustering(data, "min")
# plt.scatter(cluster.leftnode.vec[0], cluster.leftnode.vec[1], color = 'yellow')
# plt.scatter(cluster.rightnode.leftnode.vec[0], cluster.rightnode.leftnode.vec[1], color = 'red')
# plt.scatter(cluster.rightnode.rightnode.vec[0], cluster.rightnode.rightnode.vec[1], color = 'green')
# plt.scatter(df[0],df[1])
# plt.show()